In [1]:
# Install spark-related dependencies
#!wget -q  https://apache.osuosl.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!wget -q https://downloads.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
#!tar xf spark-3.5.0-bin-hadoop3.tgz
!tar xvf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
# Set up required environment variables

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=509982a6245196a939e522fa85d998739800285ffbb6b96728957322b4d8c8ac
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
import pyspark

#Creamos una sesion de spark
from pyspark.sql import SparkSession

#Creamos un objeto de spark session
spark = SparkSession.builder.appName("pysparkdf").getOrCreate()

In [3]:
#Leemos un csv
df = spark.read.option("header", "true").csv("/content/cereal.csv")

In [4]:
#Mostramos los primeros 10 registros
df.show(10)

+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+----------+
|                name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|    rating|
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+----------+
|           100% Bran|  N|   C|      70|      4|  1|   130|   10|    5|     6|   280|      25|    3|     1|0.33|68.402.973|
|   100% Natural Bran|  Q|   C|     120|      3|  5|    15|    2|    8|     8|   135|       0|    3|     1|   1|33.983.679|
|            All-Bran|  K|   C|      70|      4|  1|   260|    9|    7|     5|   320|      25|    3|     1|0.33|59.425.505|
|All-Bran with Ext...|  K|   C|      50|      4|  0|   140|   14|    8|     0|   330|      25|    3|     1| 0.5|93.704.912|
|      Almond Delight|  R|   C|     110|      2|  2|   200|    1|   14|     8|    -1|      25|    3|     1|0.75|34.384.843|
|Apple C

In [5]:
#Para ver el esquema de los datos podemos aplicar el siguiente codigo
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- mfr: string (nullable = true)
 |-- type: string (nullable = true)
 |-- calories: string (nullable = true)
 |-- protein: string (nullable = true)
 |-- fat: string (nullable = true)
 |-- sodium: string (nullable = true)
 |-- fiber: string (nullable = true)
 |-- carbo: string (nullable = true)
 |-- sugars: string (nullable = true)
 |-- potass: string (nullable = true)
 |-- vitamins: string (nullable = true)
 |-- shelf: string (nullable = true)
 |-- weight: string (nullable = true)
 |-- cups: string (nullable = true)
 |-- rating: string (nullable = true)



Comencemos con las funciones:

* **select**(): La función de selección nos ayuda a mostrar un subconjunto de columnas
seleccionadas de todo el marco de datos, solo necesitamos pasar los nombres de columna
deseados. Imprimamos tres columnas cualesquiera del marco de datos usando select ().


In [ ]:
#Seleccionamos ciertas columnas
df.select('name', 'mfr', 'rating').show(10)

+--------------------+---+----------+
|                name|mfr|    rating|
+--------------------+---+----------+
|           100% Bran|  N|68.402.973|
|   100% Natural Bran|  Q|33.983.679|
|            All-Bran|  K|59.425.505|
|All-Bran with Ext...|  K|93.704.912|
|      Almond Delight|  R|34.384.843|
|Apple Cinnamon Ch...|  G|29.509.541|
|         Apple Jacks|  K|33.174.094|
|             Basic 4|  G|37.038.562|
|           Bran Chex|  R|49.120.253|
|         Bran Flakes|  P|53.313.813|
+--------------------+---+----------+
only showing top 10 rows



* **withColumn**(): La función withColumn se usa para manipular una columna o para crear
una nueva columna con la columna existente. Es una función de transformación, también
podemos cambiar el tipo de datos de cualquier columna existente.

En el esquema de DataFrame, vimos que todas las columnas son de tipo cadena. Cambiemos el
tipo de datos de la columna de calorías a un número entero.

In [ ]:
df.withColumn("Calories",df['calories'].cast("Integer")).printSchema()

root
 |-- name: string (nullable = true)
 |-- mfr: string (nullable = true)
 |-- type: string (nullable = true)
 |-- Calories: integer (nullable = true)
 |-- protein: string (nullable = true)
 |-- fat: string (nullable = true)
 |-- sodium: string (nullable = true)
 |-- fiber: string (nullable = true)
 |-- carbo: string (nullable = true)
 |-- sugars: string (nullable = true)
 |-- potass: string (nullable = true)
 |-- vitamins: string (nullable = true)
 |-- shelf: string (nullable = true)
 |-- weight: string (nullable = true)
 |-- cups: string (nullable = true)
 |-- rating: string (nullable = true)



* **groupBy**(): La función groupBy se utiliza para recopilar los datos en grupos en
DataFrame y nos permite realizar funciones agregadas en los datos agrupados. Esta es
una operación de análisis de datos muy común similar a la cláusula groupBy en SQL.

In [6]:
df.groupBy("mfr", "calories").count().show()

+---+--------+-----+
|mfr|calories|count|
+---+--------+-----+
|  N|      70|    1|
|  R|     100|    2|
|  K|      70|    1|
|  K|      90|    2|
|  N|      80|    1|
|  R|      90|    1|
|  G|     130|    2|
|  G|     120|    1|
|  K|     100|    3|
|  P|      90|    1|
|  Q|      50|    2|
|  G|     100|    5|
|  K|     160|    1|
|  P|     110|    3|
|  K|     110|   10|
|  K|      50|    1|
|  P|     120|    3|
|  G|     110|   13|
|  R|     110|    3|
|  N|     100|    1|
+---+--------+-----+
only showing top 20 rows



* **orderBy**(): La función orderBy se utiliza para ordenar todo el marco de datos en función
de la columna particular del marco de datos. Ordena las filas del marco de datos según
los valores de las columnas. De forma predeterminada, se clasifica en orden ascendente.
Analicemos el marco de datos en función de la columna de proteínas del conjunto de datos.


In [8]:
df.orderBy("protein").show(100)
#df.groupBy("name", "Calories").count().orderBy("Calories", ascending=True).show(50)

+--------------------+--------+-----+
|                name|Calories|count|
+--------------------+--------+-----+
|          Wheat Chex|     100|    1|
|     Raisin Nut Bran|     100|    1|
|         Corn Flakes|     100|    1|
|   Grape Nuts Flakes|     100|    1|
|         Double Chex|     100|    1|
|  Quaker Oat Squares|     100|    1|
|        Golden Crisp|     100|    1|
|   Total Whole Grain|     100|    1|
| Frosted Mini-Wheats|     100|    1|
|Multi-Grain Cheerios|     100|    1|
|Cream of Wheat (Q...|     100|    1|
|          Product 19|     100|    1|
|                Life|     100|    1|
|               Maypo|     100|    1|
|Crispy Wheat & Ra...|     100|    1|
|      Quaker Oatmeal|     100|    1|
|            Wheaties|     100|    1|
|         Cocoa Puffs|     110|    1|
|  Honey Nut Cheerios|     110|    1|
|                Trix|     110|    1|
|      Fruity Pebbles|     110|    1|
|           Corn Pops|     110|    1|
|  Cracklin' Oat Bran|     110|    1|
|           

* **when**(): Esta función se utiliza  para mostrar la salida en función de la condición particular. Evalúa la condición proporcionada y luego devuelve los valores en consecuencia. Es una función SQL que admite PySpark para verificar múltiples condiciones en una secuencia y devolver el valor. Esta función funciona de manera similar como instrucciones if-then-else y switch.



In [9]:
from pyspark.sql.functions import when
#df.select("name", when(df.vitamins >= "25", "rich in vitamins")).show()
df.select("name", when(df.vitamins >= "25", "rich in vitamins").alias("vitamin_level")).show()

+--------------------+----------------+
|                name|   vitamin_level|
+--------------------+----------------+
|           100% Bran|rich in vitamins|
|   100% Natural Bran|            NULL|
|            All-Bran|rich in vitamins|
|All-Bran with Ext...|rich in vitamins|
|      Almond Delight|rich in vitamins|
|Apple Cinnamon Ch...|rich in vitamins|
|         Apple Jacks|rich in vitamins|
|             Basic 4|rich in vitamins|
|           Bran Chex|rich in vitamins|
|         Bran Flakes|rich in vitamins|
|        Cap'n'Crunch|rich in vitamins|
|            Cheerios|rich in vitamins|
|Cinnamon Toast Cr...|rich in vitamins|
|            Clusters|rich in vitamins|
|         Cocoa Puffs|rich in vitamins|
|           Corn Chex|rich in vitamins|
|         Corn Flakes|rich in vitamins|
|           Corn Pops|rich in vitamins|
|       Count Chocula|rich in vitamins|
|  Cracklin' Oat Bran|rich in vitamins|
+--------------------+----------------+
only showing top 20 rows



* **filter**(): La función de filtro se utiliza para filtrar datos en filas según los valores de
columna en particular. Por ejemplo, podemos filtrar los cereales que tienen calorías
iguales a 100.


In [11]:
from pyspark.sql.functions import filter
#df.filter(df.calories == "100").show()
df.filter(df.calories == "100").select("name", "calories").show()

+--------------------+--------+
|                name|calories|
+--------------------+--------+
|         Corn Flakes|     100|
|Cream of Wheat (Q...|     100|
|Crispy Wheat & Ra...|     100|
|         Double Chex|     100|
| Frosted Mini-Wheats|     100|
|        Golden Crisp|     100|
|   Grape Nuts Flakes|     100|
|                Life|     100|
|               Maypo|     100|
|Multi-Grain Cheerios|     100|
|          Product 19|     100|
|  Quaker Oat Squares|     100|
|      Quaker Oatmeal|     100|
|     Raisin Nut Bran|     100|
|   Total Whole Grain|     100|
|          Wheat Chex|     100|
|            Wheaties|     100|
+--------------------+--------+



* **isNull**() / **isNotNull**(): Estas dos funciones se utilizan para averiguar si hay algún valor
nulo presente en el DataFrame. Es la función más esencial para el procesamiento de datos.
Es la principal herramienta utilizada para la limpieza de datos.

In [ ]:
#isNotNull()
from pyspark.sql.functions import *

#filter data by null values
df.filter(df.name.isNotNull()).show()

+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+----------+
|                name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|    rating|
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+----------+
|           100% Bran|  N|   C|      70|      4|  1|   130|   10|    5|     6|   280|      25|    3|     1|0.33|68.402.973|
|   100% Natural Bran|  Q|   C|     120|      3|  5|    15|    2|    8|     8|   135|       0|    3|     1|   1|33.983.679|
|            All-Bran|  K|   C|      70|      4|  1|   260|    9|    7|     5|   320|      25|    3|     1|0.33|59.425.505|
|All-Bran with Ext...|  K|   C|      50|      4|  0|   140|   14|    8|     0|   330|      25|    3|     1| 0.5|93.704.912|
|      Almond Delight|  R|   C|     110|      2|  2|   200|    1|   14|     8|    -1|      25|    3|     1|0.75|34.384.843|
|Apple C

In [ ]:
#Otra manera
df.filter(df.name.isNull()).show()

+----+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+------+
|name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|rating|
+----+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+------+
+----+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+------+



Se concluye, que no hay valores nulos en el dataset. Por lo tanto, se muestra un marco de datos vacío.